In [2]:
import numpy as np
import matplotlib.pyplot as plt
def machineEpsilon(func=np.float64):
    machine_epsilon = func(1)
    while func(1)+machine_epsilon != func(1):
        machine_epsilon_last = machine_epsilon
        machine_epsilon = func(machine_epsilon) / func(2)
    return machine_epsilon_last

sqeps = np.sqrt(machineEpsilon())

In [38]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget

x = np.array([0, 25, 50, 75, 100, 125, 150, 175, 200])
y = np.array([10.6, 16.0, 45.0, 83.5, 52.8, 19.9, 10.8, 8.25, 4.7])
rho = np.array([2.3, 3.5, 4.5, 6.4, 4.4, 3.4, 2.1, 1.6, 1.1])

#starting valuse
a = np.array([60000, 70, 750])

#fitting function breit wigner
def f(x, a):
    return a[0]/((a[1] - x)**2 + a[2])

#function to calculate the chi2
def chi2(x, y, rho, a):
    return np.sum(((y - f(x, a))/rho))

#beta

def beta(chi2, x, y, a, rho, h=sqeps):
    j = np.zeros(len(a))
    ak = np.zeros(len(a))
    for i in range(len(a)):
        ak[i] = np.sum((y - f(x, a))/rho**2 * (f(x, a+ h*a[i] * np.eye(len(a))[i]) - f(x, a - h*a[i] * np.eye(len(a))[i]))/(2*h*a[i]))
    return ak

# alpha

def alpha(chi2, x, y, a, rho, h=sqeps):
    ap = np.zeros((len(a), len(a)))
    for i in range(len(a)):
        for j in range(len(a)):
            ap[i, j] = np.sum(((f(x, a + h*a[i] * np.eye(len(a))[i]) - f(x, a - h*a[i] * np.eye(len(a))[i]))/(2*h*a[i]) * (f(x, a + h*a[i] * np.eye(len(a))[j]) - f(x, a - h*a[i] * np.eye(len(a))[j]))/(2*h*a[i])/rho**2))
    return ap

#newton step

def newton_step(chi2, x, y, a, rho, h=sqeps):
    anew = a + np.linalg.pinv(alpha(chi2, x, y, a, rho, h)) @ beta(chi2, x, y, a, rho, h)
    return anew

print(alpha(chi2, x, y, a, rho))
print(beta(chi2, x, y, a, rho))


[[ 1.29459385e-07 -1.25684743e-05 -6.23420097e-06]
 [-1.25681185e-05  3.62391850e-01  4.21734690e-04]
 [-6.23420038e-06  4.21734511e-04  3.77938793e-04]]
[ 5.65729703e-04  2.66545297e+00 -1.67253945e-02]


In [39]:

chilist = []
alist = []
for i in range(3):
    a = newton_step(chi2, x, y, a, rho)
    alist.append(a)
    chilist.append(chi2(x, y, rho, a))

for i in range(100):
    a = newton_step(chi2, x, y, a, rho)
    chilist.append(chi2(x, y, rho, a))
    alist.append(a)
    if np.abs(chilist[-1] - chilist[-2]) <= np.abs(chilist[-2] * 1e-6):
        break

#return minimum chi2
print(min([abs(i) for i in chilist]))


print(alist[[abs(i) for i in chilist].index(min([abs(i) for i in chilist]))])


print(chilist)
print(a)



0.1210185262246593
[68157.06101074    77.58191749   802.28972937]
[-0.28637197547384396, 0.1210185262246593, 0.18555244270789362, 0.18566786315254924, 0.18565924222585795, 0.1856591673401463]
[68310.00240833    77.57646324   807.87147987]


In [41]:
#levenberg step
a = np.array([60000, 70, 750])
alev = []
chilev = []
def ltep(chi2, x, y, a, rho, lam, h=sqeps):
    anew = a + np.linalg.pinv(alpha(chi2, x, y, a, rho, h) + lam * np.diag(alpha(chi2, x, y, a, rho, h))) @ beta(chi2, x, y, a, rho, h)
    return anew

lam = 1.0

for i in range(60):
    anew = ltep(chi2, x, y, a, rho, lam)
    print(chi2(x, y, rho, anew))
    print(a)
    print(anew)
    print(chi2(x, y, rho, a))
    if np.abs(chi2(x, y, rho, anew)) <= np.abs(chi2(x, y, rho, a)):
        a = anew
        lam = lam/10
        print(lam)
        alev.append(a)
        chilev.append(chi2(x, y, rho, a))
    else:
        lam = lam*10
        print(lam)

print(min([abs(i) for i in chilev]))
print(alev[[abs(i) for i in chilev].index(min([abs(i) for i in chilev]))])

print(chilev)
print(alev)

-102.81839561459827
[60000    70   750]
[-1.61797693e+05  7.39793438e+01 -2.97349031e+03]
6.955739323524076
10.0
-99.50080397117075
[60000    70   750]
[-1.62256373e+05  7.39722360e+01 -2.98108011e+03]
6.955739323524076
100.0
-99.17363011850597
[60000    70   750]
[-1.62302340e+05  7.39715236e+01 -2.98184072e+03]
6.955739323524076
1000.0
-99.14095776761863
[60000    70   750]
[-1.62306937e+05  7.39714524e+01 -2.98191680e+03]
6.955739323524076
10000.0
-99.13769098181629
[60000    70   750]
[-1.62307397e+05  7.39714453e+01 -2.98192441e+03]
6.955739323524076
100000.0
-99.13736430657937
[60000    70   750]
[-1.62307443e+05  7.39714446e+01 -2.98192517e+03]
6.955739323524076
1000000.0
-99.13733119683023
[60000    70   750]
[-1.62307448e+05  7.39714445e+01 -2.98192525e+03]
6.955739323524076
10000000.0
-99.13731977781907
[60000    70   750]
[-1.62307447e+05  7.39714445e+01 -2.98192527e+03]
6.955739323524076
100000000.0
-99.13731634777861
[60000    70   750]
[-1.62307448e+05  7.39714445e+01 -2.